# Evolution of commitment and level of participation in public goods games

### Import of library needed

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Union, List

from egttools.games import AbstractNPlayerGame
from egttools import sample_simplex, calculate_nb_states
from egttools.analytical import PairwiseComparison
from egttools.utils import calculate_stationary_distribution
from egttools.plotting import draw_invasion_diagram

### Definition of the PGG class

In [1]:
class PGGwithPunishment(AbstractNPlayerGame):
    def __init__(self, 
                 group_size: int,   # number of participants in the PGG
                 c: float,          # cost of cooperation
                 r: float,          # enhancing factor (multiplier)
                 eps: float,        # cost for propose a commitment
                 F: int,            # nombre de gens qui acceptent le contrat pour pouvoir jouer le PGG  (L : Seem weird to me)
                 delta : float,     # cost to don't respect the commitment
                ):
        AbstractNPlayerGame.__init__(self, 4, group_size)
        self.nb_strategies_ = 5
        self.group_size_ = group_size

        self.strategies = ["COMP", "C", "D", "FAKE", "FREE"]  # Propose ,Cooperate, Defect, Fake, Free
        self.c = c
        self.r = r
        self.eps = eps
        self.delta = delta
        self.F = F
        self.nb_group_configurations_ = self.nb_group_configurations()
        self.calculate_payoffs()

    def get_strategy(self, strategy: str):
        """Define actions based on each strategy."""
        
        contribute = False
        propose = False
        accept = False
        
        if strategy == "COMP": # Proposers 
            contribute = True
            propose = True
            accept = True
        elif strategy == "C": # Unconditional contributors
            contribute = True
            propose = False
            accept = True
        elif strategy == "D": # Defectors
            contribute = False
            propose = False
            accept = False
        elif strategy == "FAKE": # Fake committers
            contribute = False
            propose = False
            accept = True
        elif strategy == "FREE": # commitment free-riders
            pass
        
        return contribute, propose, accept

    def play(self, group_composition: Union[List[int], np.ndarray], game_payoffs: np.ndarray) -> None:
        PROPOSER, COOPERATOR, DEFECTOR, FAKER, FREE  = 0, 1, 2, 3, 4
        nb_contributors = group_composition[PROPOSER] + group_composition[COOPERATOR]
        nb_acceptors = group_composition[FAKER] + nb_contributors


        # There is no proposer so it is a original PGG
        if group_composition[PROPOSER] == 0 : # Node 5
            nb_contributors = group_composition[COOPERATOR]
            nb_defectors = group_composition[FAKER] + group_composition[DEFECTOR] + group_composition[FREE]
            nb_participants = nb_defectors + nb_contributors
            total_reward = self.r * self.c * nb_contributors 
            individual_reward = total_reward / nb_participants 

            for index, _ in enumerate(group_composition):
                contributes, proposes, accepts = self.get_strategy(self.strategies[index])
                game_payoffs[index] += individual_reward
                if contributes :
                    game_payoffs[index] -= self.c 

        # There is at least one proposer 
        elif group_composition[PROPOSER] >= 1 : # Node 2
            if nb_acceptors < self.F :
                pass
                # Code van Eliott 

            else : 
                return # The PGG is not played
        
        # The PGG is not played
        else :
            return


    def calculate_payoffs(self) -> np.ndarray:
        """Calculate and store the payoffs for each strategy in the game."""
        
        payoffs_container = np.zeros(shape=(self.nb_strategies_,), dtype=np.float64)
        
        for i in range(self.nb_group_configurations_):
            # Generate a sample group composition
            group_composition = sample_simplex(i, self.group_size_, self.nb_strategies_)
            group_composition = np.array(group_composition, dtype=float)

            # Play the game with the given group composition
            #self.play(group_composition, payoffs_container)
            self.play(group_composition, payoffs_container)

            # Store payoffs for each strategy
            for strategy_index, strategy_payoff in enumerate(payoffs_container):
                self.update_payoff(strategy_index, i, strategy_payoff)
                
            # Reset the payoff container for the next configuration
            payoffs_container[:] = 0

        return self.payoffs()

NameError: name 'AbstractNPlayerGame' is not defined